In [ ]:
"""
cell dedicated to google drive if it will be necessary a GPU
"""

In [57]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import os 
import cv2 
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical 
from keras import backend as K
from sklearn.model_selection import train_test_split


from keras.layers import Input, Conv2D, BatchNormalization, Activation, Flatten, Dense, Dropout, concatenate, Lambda
from tensorflow.keras.initializers import RandomNormal
from keras.models import Sequential,Model
import time
from keras.optimizers import Adam
import seaborn as sns
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D

from keras.regularizers import l2


from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras.optimizers import RMSprop

In [35]:
train_data  = np.load('training_data.npy')
train_label = np.load('training_labe.npy')

## MODEL GENERATION 

Here we make feature generation network (siamese network) to process image into features. The network starts off randomly initialized and will be trained to generate useful vector features from input images (hopefully). 


In [49]:
def siamese_model(input_shape): 
    
    
    # define two inputs 
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    model = Sequential()
    
    model.add(Conv2D(64, (10,10), activation='relu',kernel_initializer = 'random_normal',bias_initializer='random_normal',  
              input_shape=input_shape, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    
    
    model.add(Conv2D(128, (7,7), activation='relu',  kernel_initializer = 'random_normal',bias_initializer='random_normal',
              input_shape=input_shape, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    

    model.add(Conv2D(128, (4,4), activation='relu',  kernel_initializer = 'random_normal',bias_initializer='random_normal',
              input_shape=input_shape, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    

    model.add(Conv2D(256, (4,4), activation='relu', kernel_initializer = 'random_normal',bias_initializer='random_normal',
              input_shape=input_shape, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    
    
    model.add(Flatten())
    
    
    model.add(Dense(4096, activation='sigmoid',kernel_regularizer=l2(1e-3))) 
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    
    combined_features = concatenate([encoded_l, encoded_r], name = 'merge_features')
    

    combined_features = Dense(16, activation = 'linear')(combined_features)
    combined_features = BatchNormalization()(combined_features)
    combined_features = Activation('relu')(combined_features)
    combined_features = Dense(4, activation = 'linear')(combined_features)
    combined_features = BatchNormalization()(combined_features)
    combined_features = Activation('relu')(combined_features)
    combined_features = Dense(1, activation = 'sigmoid')(combined_features)
    similarity_model = Model(inputs = [left_input, right_input], outputs = [combined_features], name = 'Similarity_Model')
    #similarity_model.summary()


    
    
    

    # Add a customized layer to compute the absolute difference between the encodings
    #L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    #L1_distance = L1_layer([encoded_l, encoded_r])
    
    

    # Add a dense layer with a sigmoid unit to generate the similarity score
    #prediction = Dense(1,activation='sigmoid')(L1_distance)
    

        # Connect the inputs with the outputs
    #siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return similarity_model   

In [50]:
input_dim = train_data.shape[2:]
print(input_dim)
base_network = siamese_model(input_dim)

(100, 100, 3)


In [51]:
# setup the optimization process
base_network.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['mae'])

In [53]:
x_train, x_test, y_train, y_test = train_test_split(train_data, train_label, test_size=.15)

In [54]:
print('x_train',x_train.shape)
print('x_test',x_test.shape)
print('y_train',y_train.shape)
print('y_test',y_test.shape)

x_train (5100, 2, 100, 100, 3)
x_test (900, 2, 100, 100, 3)
y_train (5100, 1)
y_test (900, 1)


In [58]:
epochs = 20
rms = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)#RMSprop()
rms = RMSprop()

earlyStopping = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=3,
                              verbose=1,
                              restore_best_weights=True)
callback_early_stop_reduceLROnPlateau=[earlyStopping]

In [60]:
base_network.compile(loss="binary_crossentropy", optimizer=rms,metrics=["binary_accuracy"])
base_network.summary()

Model: "Similarity_Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
sequential_11 (Sequential)      (None, 4096)         5406016     input_21[0][0]                   
                                                                 input_22[0][0]                   
__________________________________________________________________________________________________
merge_features (Concatenate)    (None, 8192)         0           sequential_11[1][0

In [ ]:
img_1 = x_train[:, 0]
img2 = x_train[:, 1]

history = base_network.fit([img_1, img2], y_train, validation_split=.20,batch_size= 256, verbose=1, epochs=20)

Train on 4080 samples, validate on 1020 samples
Epoch 1/20
